In [5]:
import pandas as pd
import time, datetime
import re
import pickle

from datetime import timedelta
from twitterscraper import query_tweets
from nltk.tokenize import RegexpTokenizer

___

In [2]:
def scrape_twitter(place):
    # set empty lists that we will fill with tweet data
    text = []
    locs = []
    times = []

    # scrape twitter for tweets containing certain keywords 
    query_string = f'"power outage" OR "power is out" OR "power\'s out" OR "blackouts" OR "blackout" -"video game" OR "power failure" OR "power failures" OR "no electricity" OR "power shortage" OR "electrical failure" OR "power loss" OR "power cuts" OR "power cut" OR "power went out" OR "power interuption" OR "brownout" OR "power goes out" OR "brownouts" OR "without power" near:"{place}" within:15mi -filter:retweets'
    list_of_tweets = query_tweets(query_string,
                            begindate = datetime.datetime.today().date(),
                            enddate = datetime.datetime.today().date() + timedelta(days=1),
                            poolsize = 2,
                            lang="en"
                           )

    # loop through each tweet to grab data and append the data to their respective lists
    for tweet in list_of_tweets:
        locs.append(place)
        text.append(tweet.text)
        times.append(tweet.timestamp)

    # build the dataframe 
    df = pd.DataFrame({
        'tweet': text,
        'location': locs,
        'time_stamp': times,
    })

    return df

In [3]:
df = scrape_twitter('Alabama')

INFO: queries: ['"power outage" OR "power is out" OR "power\'s out" OR "blackouts" OR "blackout" -"video game" OR "power failure" OR "power failures" OR "no electricity" OR "power shortage" OR "electrical failure" OR "power loss" OR "power cuts" OR "power cut" OR "power went out" OR "power interuption" OR "brownout" OR "power goes out" OR "brownouts" OR "without power" near:"Alabama" within:15mi -filter:retweets since:2019-11-06 until:2019-11-07']
INFO: Querying "power outage" OR "power is out" OR "power's out" OR "blackouts" OR "blackout" -"video game" OR "power failure" OR "power failures" OR "no electricity" OR "power shortage" OR "electrical failure" OR "power loss" OR "power cuts" OR "power cut" OR "power went out" OR "power interuption" OR "brownout" OR "power goes out" OR "brownouts" OR "without power" near:"Alabama" within:15mi -filter:retweets since:2019-11-06 until:2019-11-07
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q="power%20outage"%20

In [4]:
# remove any twitter pic urls
df['tweet'] = [re.sub(r'pic.twitter.com\S+', '', post).strip() for post in df['tweet']]

# remove any http urls
df['tweet'] = [re.sub(r'http\S+', '', post).strip() for post in df['tweet']]

# instatiate the tokenizer
tknr = RegexpTokenizer(r'[a-zA-Z&0-9]+')

# start with empty lists
tokens = []

# fill the list with tokenized versions of each post title
for post in df['tweet']:
    tokens.append(" ".join(tknr.tokenize(post.lower())))

df['tweet'] = tokens

df = df.reset_index(drop=True)

tweets = [tweet for tweet in df['tweet']]

master_tweets = ' '.join(tweets)

master_tweets

'love 2 have a random power outage and suddenly have a desktop with a fucked hard drive guess i ll scrape together some money to fix it bungie destinythegame i now refuse to play any activity whatsoever that has blackout as a modifier remove that mess from the game completely love 2 have a random power outage and suddenly have a desktop with a fucked hard drive guess i ll scrape together some money to fix it bungie destinythegame i now refuse to play any activity whatsoever that has blackout as a modifier remove that mess from the game completely'